In [2]:
import numpy as np
import pandas as pd
import sys
import csv
import pandas as pd
import pickle
import os
import io
from datetime import datetime
from collections import defaultdict

In [3]:
compensation = pd.read_csv("Raw Data/Lobbyist_Data_-_Compensation.csv")
contributions = pd.read_csv("Raw Data/Lobbyist_Data_-_Contributions (1).csv")
contributions_ward = pd.read_csv("Raw Data/contributions_with_ward_data.csv")
employers = pd.read_csv("Raw Data/Lobbyist_Data_-_Employers.csv")
expenditures_large = pd.read_csv("Raw Data/Lobbyist_Data_-_Expenditures_-_Large.csv")
expenditures_small = pd.read_csv("Raw Data/Lobbyist_Data_-_Expenditures_-_Small.csv")
gifts = pd.read_csv("Raw Data/Lobbyist_Data_-_Gifts (1).csv")
activity = pd.read_csv("Raw Data/Lobbyist_Data_-_Lobbying_Activity (1).csv")
empl_contr = pd.read_csv("Raw Data/Lobbyist_Data_-_Lobbyist__Employer__Client_Combinations (1).csv")
lobbyists = pd.read_csv("Raw Data/Lobbyist_Data_-_Lobbyists (1).csv")
wards = pd.read_csv("Raw Data/Ward_Offices.csv")

In [4]:
for i in contributions["RECIPIENT"].unique():
    print(i)

ALDERMAN MARIA HADDEN
24TH WARD ORGANIZATION - ALD. MICHAEL SCOTT
ALD. GILBERT VILLEGAS
nan
N?A
JASON ERVIN
VILLEGAS FOR COMMITTEEPERSON
FRIENDS OF MICHAEL RODRIGUEZ
THE NEW 37TH WARD DEMOCRATIC ORG
NEIGHBORS FOR DANIEL LASPATA
FRIENDS OF GILBERT VILLEGAS
FRIENDS OF RODERICK SAWYER
FRIENDS OF LESLIE A HAIRSTON
ALDERMAN RODERICK SAWYER
ALDERMAN BYRON SIGCHO LOPEZ
CITIZENS FOR JASON C. ERVIN
DEBRA SILVERSTEIN
LESLIE HAIRSTON
TOM TUNNEY
8TH WARD DEMS
11TH WARD DEMS
19TH WARD DEMOCRATIC ORGANIZATION
FRIENDS OF RODRIGUEZ
FRIENDS OF MICHELLE A. HARRIS
EMMA MITTS
FRIENDS FOR DEBRA SILVERSTEIN
ALDERMAN HOWARD BROOKINS
FRIENDS OF SUSAN SADLOWSKI GARZA
HOPKINS FOR CHICAGO
FRIENDS OF NUGGENT
COMMITTEE TO ELECT HOWARD B BROOKINS, JR
CITIZENS TO ELECT FELIX CARDONA JR
CITIZENS FOR ERVIN
ALDERMAN JASON ERVIN
COMM TO ELECT BROOKINS
24TH WARD DEMS
NEIBORS FOR ANDRE
NEIGHBORS FOR LASPATA
28TH WARD DEMOCRATIC ORGANIZATION
MICHAEL SCOTT - 24TH WARD ALDERMAN
GILBERT VILLEGAS
MATT MARTIN
ANNA VALENCIA
BRIA

In [5]:
contributions_ward = contributions_ward[contributions_ward.Ward != 0]
contributions_ward.head(10)

,Unnamed: 0,CONTRIBUTION_ID,PERIOD_START,PERIOD_END,CONTRIBUTION_DATE,RECIPIENT,AMOUNT,LOBBYIST_ID,LOBBYIST_FIRST_NAME,LOBBYIST_LAST_NAME,CREATED_DATE,Ward,ID
0,0,2240204276,07/01/2020,09/30/2020,08/24/2020,ALDERMAN MARIA HADDEN,250.0,23981,ALEXANDRA,SIMS,12/01/2020,49,95040.0
1,1,16680854,04/01/2020,06/30/2020,06/10/2020,24TH WARD ORGANIZATION - ALD. MICHAEL SCOTT,500.0,16501,REYAHD,KAZMI,12/10/2020,24,264924.0
2,2,146026078,01/01/2020,03/31/2020,02/18/2020,ALD. GILBERT VILLEGAS,150.0,16501,REYAHD,KAZMI,12/10/2020,36,70359.0
5,5,752575625,10/01/2020,12/31/2020,10/19/2020,JASON ERVIN,500.0,18121,PETER,SKOSEY,12/18/2020,28,308183.0
7,7,399395029,07/01/2020,09/30/2020,09/03/2020,VILLEGAS FOR COMMITTEEPERSON,1500.0,4126,MICHAEL,ALVAREZ,12/20/2020,36,70359.0
8,8,2229231542,01/01/2020,03/31/2020,03/12/2020,FRIENDS OF MICHAEL RODRIGUEZ,1000.0,4126,MICHAEL,ALVAREZ,12/20/2020,22,77908.0
11,11,1892676832,01/01/2020,03/31/2020,02/11/2020,FRIENDS OF GILBERT VILLEGAS,1500.0,4126,MICHAEL,ALVAREZ,12/20/2020,36,70359.0
12,12,4231410080,01/01/2020,03/31/2020,03/12/2020,FRIENDS OF RODERICK SAWYER,1500.0,4126,MICHAEL,ALVAREZ,12/20/2020,6,219806.0
13,13,691794209,07/01/2020,09/30/2020,09/18/2020,FRIENDS OF LESLIE A HAIRSTON,1500.0,4126,MICHAEL,ALVAREZ,12/20/2020,5,159667.0
14,14,915701728,10/01/2020,12/31/2020,12/21/2020,ALDERMAN RODERICK SAWYER,250.0,20567,LINDSAY,ANDERSON,12/29/2020,6,219806.0


In [1]:
#This is the main analysis function for the data.
#It creates a dictionary of the form {streamer1: {streamer2: overlap, streamer3: overlap}}
#This allows us to have an integer of overlapping viewers for each streamer with every other streamer
def CreateOverlapDict(dictionary):
    viewerOverlapDict = {}
    count = 1
    completedStreamers = [] #Save which streamers have been processed to avoid repeating
    for key in dictionary:
        dictionary[key] = set(dictionary[key]) #Make viewer list a set to dramatically decrease comparison time
    for key in dictionary:
        tempList = {}

        totalLength = len(dictionary.keys())
        print(str(count) + "/" + str(totalLength)) #Print progress so I can keep track

        for comparisonKey in dictionary: #Loop through every key again for each key in the dictionary
            if(comparisonKey != key and comparisonKey not in completedStreamers): #If its not a self comparison and the comparison hasn't already been completed
                overlapSize = len(dictionary[key] & dictionary[comparisonKey]) #Find the overlap size of the two streamers using set intersection
                if(overlapSize > 2 ):
                    tempList[comparisonKey] = overlapSize #If the size is over 2 add {comparisonStreamer: overlap} to the dictionary
        viewerOverlapDict[key] = tempList #Add this comparison dictionary to the larger dictionary for that streamer
        completedStreamers.append(key) #Add the streamer to completed as no comparisons using this streamer need to be done anymore
        count+=1
    return viewerOverlapDict

#Generates a new csv file for the edge list on Gephi
def GenerateGephiData(dict, graph_type):
    fileString = "GephiData/"+graph_type+"_EDGELIST.csv"
    with open(fileString, 'w') as csvfile:
        writer = csv.writer(csvfile)
        #writer.writeheader()
        writer.writerow(["Source", "Target", "Weight"]) #These column headers are used in Gephi automatically
        for key, value in dict.items():
            nodeA = key
            for node, count in value.items():
                nodeB = node
                print(str(nodeA) + " " + str(nodeB))
                writer.writerow([str(nodeA), str(nodeB), count]) #nodeA is streamer1, nodeB is streamer2, and count is their overlapping viewers

#Generates a new csv file for the node list labels on Gephi
def GenerateGephiLabels(rawDict, graph_type):
    fileString = "GephiData/"+graph_type+"_LABELS.csv"
    print("Generating Labels...")
    sys.stdout.flush()
    with open(fileString, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["ID", "Label", "Count of Contributions"]) #These columns are used in Gephi automatically
        for key, value in rawDict.items():
            if graph_type.split("-")[0] == "lobbyists":
                lobbyist_name = lobbyists[lobbyists["LOBBYIST_ID"] == key].iloc[0]["FIRST_NAME"]+" "+lobbyists[lobbyists["LOBBYIST_ID"] == key].iloc[0]["LAST_NAME"]
                writer.writerow([key, lobbyist_name, len(value)]) #This data is streamer1, streamer1, and # of unique viewers for streamer1
            else:
                writer.writerow([key, key, len(value)]) #This data is streamer1, streamer1, and # of unique viewers for streamer1

## Nodes: Lobbyists, Edges: Companies

In [51]:
comp_dict = defaultdict(list)

for idx,row in compensation.iterrows():
    comp_dict[row['LOBBYIST_ID']].append(row['CLIENT_ID'])

#Generate Gephi data files with the dictionaries
ovlp_dict = CreateOverlapDict(comp_dict) #Process data creating dictionary of {streamer1: {streamer2: overlap, streamer3: overlap}}
GenerateGephiData(ovlp_dict, "lobbyists-companies")
GenerateGephiLabels(comp_dict, "lobbyists-companies")

1/1198
2/1198
3/1198
4/1198
5/1198
6/1198
7/1198
8/1198
9/1198
10/1198
11/1198
12/1198
13/1198
14/1198
15/1198
16/1198
17/1198
18/1198
19/1198
20/1198
21/1198
22/1198
23/1198
24/1198
25/1198
26/1198
27/1198
28/1198
29/1198
30/1198
31/1198
32/1198
33/1198
34/1198
35/1198
36/1198
37/1198
38/1198
39/1198
40/1198
41/1198
42/1198
43/1198
44/1198
45/1198
46/1198
47/1198
48/1198
49/1198
50/1198
51/1198
52/1198
53/1198
54/1198
55/1198
56/1198
57/1198
58/1198
59/1198
60/1198
61/1198
62/1198
63/1198
64/1198
65/1198
66/1198
67/1198
68/1198
69/1198
70/1198
71/1198
72/1198
73/1198
74/1198
75/1198
76/1198
77/1198
78/1198
79/1198
80/1198
81/1198
82/1198
83/1198
84/1198
85/1198
86/1198
87/1198
88/1198
89/1198
90/1198
91/1198
92/1198
93/1198
94/1198
95/1198
96/1198
97/1198
98/1198
99/1198
100/1198
101/1198
102/1198
103/1198
104/1198
105/1198
106/1198
107/1198
108/1198
109/1198
110/1198
111/1198
112/1198
113/1198
114/1198
115/1198
116/1198
117/1198
118/1198
119/1198
120/1198
121/1198
122/1198
123/1198
1

931/1198
932/1198
933/1198
934/1198
935/1198
936/1198
937/1198
938/1198
939/1198
940/1198
941/1198
942/1198
943/1198
944/1198
945/1198
946/1198
947/1198
948/1198
949/1198
950/1198
951/1198
952/1198
953/1198
954/1198
955/1198
956/1198
957/1198
958/1198
959/1198
960/1198
961/1198
962/1198
963/1198
964/1198
965/1198
966/1198
967/1198
968/1198
969/1198
970/1198
971/1198
972/1198
973/1198
974/1198
975/1198
976/1198
977/1198
978/1198
979/1198
980/1198
981/1198
982/1198
983/1198
984/1198
985/1198
986/1198
987/1198
988/1198
989/1198
990/1198
991/1198
992/1198
993/1198
994/1198
995/1198
996/1198
997/1198
998/1198
999/1198
1000/1198
1001/1198
1002/1198
1003/1198
1004/1198
1005/1198
1006/1198
1007/1198
1008/1198
1009/1198
1010/1198
1011/1198
1012/1198
1013/1198
1014/1198
1015/1198
1016/1198
1017/1198
1018/1198
1019/1198
1020/1198
1021/1198
1022/1198
1023/1198
1024/1198
1025/1198
1026/1198
1027/1198
1028/1198
1029/1198
1030/1198
1031/1198
1032/1198
1033/1198
1034/1198
1035/1198
1036/1198
1037/1198

# Contributions

## Nodes: Recipient, Edges: Lobbyists

In [55]:
contr_dict = defaultdict(list)

for idx,row in contributions_ward.iterrows():
    contr_dict[row['RECIPIENT']].append(row['LOBBYIST_ID'])

#Generate Gephi data files with the dictionaries
ovlp_dict = CreateOverlapDict(contr_dict) #Process data creating dictionary of {streamer1: {streamer2: overlap, streamer3: overlap}}
GenerateGephiData(ovlp_dict, "recipients-lobbyists")
GenerateGephiLabels(contr_dict, "recipients-lobbyists")

1/667
2/667
3/667
4/667
5/667
6/667
7/667
8/667
9/667
10/667
11/667
12/667
13/667
14/667
15/667
16/667
17/667
18/667
19/667
20/667
21/667
22/667
23/667
24/667
25/667
26/667
27/667
28/667
29/667
30/667
31/667
32/667
33/667
34/667
35/667
36/667
37/667
38/667
39/667
40/667
41/667
42/667
43/667
44/667
45/667
46/667
47/667
48/667
49/667
50/667
51/667
52/667
53/667
54/667
55/667
56/667
57/667
58/667
59/667
60/667
61/667
62/667
63/667
64/667
65/667
66/667
67/667
68/667
69/667
70/667
71/667
72/667
73/667
74/667
75/667
76/667
77/667
78/667
79/667
80/667
81/667
82/667
83/667
84/667
85/667
86/667
87/667
88/667
89/667
90/667
91/667
92/667
93/667
94/667
95/667
96/667
97/667
98/667
99/667
100/667
101/667
102/667
103/667
104/667
105/667
106/667
107/667
108/667
109/667
110/667
111/667
112/667
113/667
114/667
115/667
116/667
117/667
118/667
119/667
120/667
121/667
122/667
123/667
124/667
125/667
126/667
127/667
128/667
129/667
130/667
131/667
132/667
133/667
134/667
135/667
136/667
137/667
138/667
139/

## Nodes: Lobbyists, Edges: Recipient NO OVERLAP

In [30]:
# contr_dict = defaultdict(list)

# for idx,row in contributions.iterrows():
#     contr_dict[row['RECIPIENT']].append(row['LOBBYIST_ID'])

# #Generate Gephi data files with the dictionaries
# ovlp_dict = CreateOverlapDict(comp_dict) #Process data creating dictionary of {streamer1: {streamer2: overlap, streamer3: overlap}}
# GenerateGephiData(ovlp_dict, "lobbyists-recipients")
# GenerateGephiLabels(comp_dict, "lobbyists-recipients")

Generating Labels...


## Nodes: Ward, Edges: Lobbyists

In [53]:
contr_dict = defaultdict(list)

for idx,row in contributions_ward.iterrows():
    contr_dict[row['Ward']].append(row['LOBBYIST_ID'])

#Generate Gephi data files with the dictionaries
ovlp_dict = CreateOverlapDict(contr_dict) #Process data creating dictionary of {streamer1: {streamer2: overlap, streamer3: overlap}}
GenerateGephiData(ovlp_dict, "ward-lobbyists")
GenerateGephiLabels(contr_dict, "ward-lobbyists")

1/48
2/48
3/48
4/48
5/48
6/48
7/48
8/48
9/48
10/48
11/48
12/48
13/48
14/48
15/48
16/48
17/48
18/48
19/48
20/48
21/48
22/48
23/48
24/48
25/48
26/48
27/48
28/48
29/48
30/48
31/48
32/48
33/48
34/48
35/48
36/48
37/48
38/48
39/48
40/48
41/48
42/48
43/48
44/48
45/48
46/48
47/48
48/48
24 36
24 28
24 22
24 6
24 5
24 15
24 50
24 44
24 8
24 37
24 21
24 10
24 2
24 31
24 47
24 27
24 41
24 16
24 29
24 39
24 34
24 42
24 3
24 40
24 32
24 9
24 26
24 4
24 43
24 19
24 48
24 1
24 17
24 30
24 12
24 11
24 38
24 46
24 14
24 18
24 7
24 13
24 23
36 28
36 22
36 6
36 5
36 15
36 50
36 44
36 8
36 37
36 21
36 10
36 2
36 31
36 47
36 27
36 41
36 16
36 29
36 39
36 34
36 42
36 3
36 40
36 32
36 9
36 26
36 4
36 43
36 19
36 48
36 1
36 17
36 30
36 12
36 11
36 38
36 46
36 14
36 18
36 7
36 23
28 22
28 6
28 5
28 15
28 50
28 44
28 8
28 37
28 21
28 10
28 2
28 31
28 47
28 27
28 41
28 16
28 29
28 39
28 34
28 42
28 3
28 40
28 32
28 9
28 26
28 4
28 43
28 19
28 48
28 1
28 17
28 30
28 12
28 11
28 38
28 46
28 14
28 18
28 7
28 23
22 6

In [46]:
a = compensation[(compensation["LOBBYIST_FIRST_NAME"]=="DAVID") & (compensation["LOBBYIST_LAST_NAME"]=="DRING")]["CLIENT_NAME"].unique()
b = compensation[(compensation["LOBBYIST_FIRST_NAME"]=="COURTNEY") & (compensation["LOBBYIST_LAST_NAME"]=="NOTTAGE")]["CLIENT_NAME"].unique()
c = compensation[(compensation["LOBBYIST_FIRST_NAME"]=="MICHAEL") & (compensation["LOBBYIST_LAST_NAME"]=="KASPER")]["CLIENT_NAME"].unique()
#d = compensation[(compensation["LOBBYIST_FIRST_NAME"]=="CARL") & (compensation["LOBBYIST_LAST_NAME"]=="RICE")]["CLIENT_NAME"].unique()


a_b = [i for i in a if i in b]
a_b_c = [i for i in a_b if i in c]
#a_b_c_d = [i for i in a_b_c if i in d]
len(a_b_c)#_d

57

In [56]:
ovlp_dict

{'ALDERMAN MARIA HADDEN': {},
 '24TH WARD ORGANIZATION - ALD. MICHAEL SCOTT': {},
 'ALD. GILBERT VILLEGAS': {},
 'JASON ERVIN': {'FRIENDS OF WALTER BURNETT': 3,
  'FRIENDS OF LESLIE HAIRSTON': 3,
  'FRIENDS OF ED BURKE': 3,
  'CARRIE AUSTIN': 3},
 'VILLEGAS FOR COMMITTEEPERSON': {'HOPKINS FOR CHICAGO': 3},
 'FRIENDS OF MICHAEL RODRIGUEZ': {'FRIENDS OF GILBERT VILLEGAS': 3,
  'HOPKINS FOR CHICAGO': 3,
  'CITIZENS FOR ERVIN': 3,
  'CITIZENS FOR WAGUESPACK': 4,
  'CITIZENS FOR JOE MOORE': 3},
 'FRIENDS OF GILBERT VILLEGAS': {'FRIENDS OF RODERICK SAWYER': 5,
  'FRIENDS OF MICHELLE A. HARRIS': 3,
  'FRIENDS FOR DEBRA SILVERSTEIN': 5,
  'FRIENDS OF SUSAN SADLOWSKI GARZA': 4,
  'HOPKINS FOR CHICAGO': 11,
  'CITIZENS FOR ERVIN': 6,
  'FRIENDS OF WALTER BURNETT, JR.': 4,
  'FRIENDS OF ANTHONY NAPOLITANO': 3,
  'FRIENDS OF SAMANTHA NUGENT': 6,
  'FRIENDS OF CARRIE AUSTIN': 5,
  'FRIENDS OF LESLIE A. HAIRSTON': 5,
  'CITIZENS FOR REILLY': 5,
  'CITIZENS FOR PAT DOWELL': 6,
  'CITIZENS FOR MICHELL